# Widzenie maszynowe
## Laboratorium 7 - Estymacja pozy obiektu
*Autor: Paweł Mendroch* - [Github](https://github.com/FrozenTear7/computer-vision-lab/tree/master/lab7)

## Setup

In [1]:
import cv2, numpy as np
import os
import glob
import plyfile
import random

In [2]:
camera_matrix =  np.array(
    [
        [640.,   0., 320.],
        [  0., 640., 240.],
        [  0.,   0.,   1.]
    ]
)

distortion_coeffs = np.zeros(4)

## Odczytanie modelu

In [3]:
data = plyfile.PlyData.read("./drill.ply")["vertex"]
model_points = np.c_[data["x"], data["y"], data["z"]]
model_points = model_points[::100]
model_points = model_points.astype(np.float32)

## Odczytanie parametrów wiertarki

In [4]:
dataset_dir = "./dataset/drill/"
angle = "angle 0/"
dataset_subdir = "scenario 1/rgba/"
drill_file = "00005"
results_dir = "./results/"

image = cv2.imread(f"{dataset_dir}{angle}{dataset_subdir}{drill_file}.png")
mask = cv2.imread(f"{dataset_dir}{angle}{dataset_subdir}{drill_file}-mask.png")
boundary = cv2.imread(f"{dataset_dir}{angle}{dataset_subdir}{drill_file}-boundary.png")
attributes = open(f"{dataset_dir}{angle}{dataset_subdir}{drill_file}-attributes.txt")
attributes = attributes.readlines()
rotation = np.array(list(map(lambda x: x + np.random.uniform(-0.1, 0.1), np.array(attributes[5].split()[1:]).astype(np.float))))
translation = np.array(list(map(lambda x: x + np.random.uniform(-0.1, 0.1), np.array(attributes[6].split()[1:]).astype(np.float))))

In [5]:
projected_points, _ = cv2.projectPoints(model_points, rotation, translation, camera_matrix, distortion_coeffs)

projected_points = projected_points.astype(np.int)
projected_points = projected_points.reshape((-1, 2))
for point in projected_points:
    image[point[1], point[0]] = [255, 255, 255]

In [6]:
cv2.imwrite(f"{drill_file}-result.png", image)

True

## Fitness

In [7]:
segmentation_pixels = 0

for i in range(len(mask)):
    for j in range(len(mask[i])):
        if mask[i, j].all() != 0:
            segmentation_pixels += 1

In [8]:
def fitness_function(particle):
    particle_hits = 0

    for point in particle:
        if mask[int(point[1]), int(point[0])].all() == 0:
            particle_hits += 1

    return 0.5 * particle_hits / len(particle) + 0.5 * ((segmentation_pixels - particle_hits) / segmentation_pixels)

## Szkielet PSO

In [9]:
W = 0.729
c1 = 2.05
c2 = 2.05
target = 0.4
target_error = 0.05
learning_rate = 0.25 # Zwalniam learning rate inaczej dochodzi do zbiegania się punktów cząsteczki

n_iterations = 10
n_points = len(model_points)
n_particles = 30
# n_particles = 50

particle_position_vector = []

# Element losowości rotacji i translacji
for i in range(n_particles):
    particle_rotation = np.array(list(map(lambda x: x * np.random.uniform(0.4, 0.6), rotation)))
    particle_translation = np.array(list(map(lambda x: x * np.random.uniform(0.4, 0.6), translation)))

    particle_pos, _ = cv2.projectPoints(model_points, particle_rotation, particle_translation, camera_matrix, distortion_coeffs)
    particle_pos = particle_pos.astype(np.int).reshape((-1, 2))

    particle_position_vector.append(particle_pos)

pbest_position = particle_position_vector
gbest_position = None

pbest_fitness_value = np.full(n_particles, np.inf)
gbest_fitness_value = np.inf

gbest_index = 0

velocity_vector = np.zeros((n_particles, n_points, 2))

iteration = 0
while iteration < n_iterations:
    for i in range(n_particles):
        fitness_cadidate = fitness_function(particle_position_vector[i])

        if(pbest_fitness_value[i] > fitness_cadidate): 
            pbest_fitness_value[i] = fitness_cadidate
            pbest_position[i] = particle_position_vector[i]

        if(gbest_fitness_value > fitness_cadidate):
            gbest_fitness_value = fitness_cadidate
            gbest_position = particle_position_vector[i]
            gbest_index = i
        
    if(abs(gbest_fitness_value - target) < target_error):
        break

    for i in range(n_particles):
        velocity_vector[i] = (W * velocity_vector[i]) + (c1 * random.random()) * (pbest_position[i] - particle_position_vector[i]) + (c2 * random.random()) * (gbest_position - particle_position_vector[i])

        new_position = learning_rate * velocity_vector[i] + particle_position_vector[i]
        new_position[:, 0] %= 640
        new_position[:, 1] %= 480
        particle_position_vector[i] = new_position

    print(f"Iteration: {iteration}, fitness: {gbest_fitness_value}")

    # Zapisuję ostatnią iterację
    if iteration == n_iterations - 1:
        result_image = np.copy(image)
    
        for point in particle_position_vector[gbest_index]:
            result_image[int(point[1]), int(point[0])] = [0, 255, 0]

        cv2.imwrite(f"{results_dir}{drill_file}_{n_particles}_{angle[6:-1]}_result.png", result_image)
        fitnessChanged = False

    iteration = iteration + 1

Iteration: 0, fitness: 0.6214775861534166
Iteration: 1, fitness: 0.621382755875389
Iteration: 2, fitness: 0.6199128865659604
Iteration: 3, fitness: 0.6133695973820528
Iteration: 4, fitness: 0.6036969090232327
Iteration: 5, fitness: 0.569700254350321
Iteration: 6, fitness: 0.5640104376686621
Iteration: 7, fitness: 0.5467987422066442
Iteration: 8, fitness: 0.5467987422066442
Iteration: 9, fitness: 0.5467987422066442


## Wnioski

Poniżej przedstawiam wyniki dla 4 kątów dla 30 cząsteczek:

![](./results/00005_30_0_result.png)
![](./results/00005_30_30_result.png)
![](./results/00005_30_60_result.png)
![](./results/00005_30_90_result.png)

A poniżej dla 50 cząsteczek:

![](./results/00005_50_0_result.png)
![](./results/00005_50_30_result.png)
![](./results/00005_50_60_result.png)
![](./results/00005_50_90_result.png)